In [2]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as file_handle:
    text = file_handle.read()

chars = sorted(set(text))
chars

['\n',
 ' ',
 '!',
 '"',
 '&',
 "'",
 '(',
 ')',
 '*',
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '[',
 ']',
 '_',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '\ufeff']

In [3]:
str_to_int = { ch:i for i, ch in enumerate(chars)}
int_to_ch = { i:ch for i, ch in enumerate(chars)}

encode = lambda s: [str_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_ch[n] for n in l])

encoded_hello = encode('hello')
decoded_hello = decode(encoded_hello)
print(decoded_hello)

hello


In [4]:
import torch

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
BLOCK_SIZE = 8
BATCH_SIZE = 4

In [5]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,  1, 47,
        33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0, 26, 49,  0,  0, 36,
        11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,  0,  0, 25, 45, 44, 32,
        39, 42,  1, 39, 30,  1, 44, 32, 29,  1, 47, 33, 50, 25, 42, 28,  1, 39,
        30,  1, 39, 50,  9,  1, 44, 32, 29,  1, 36, 25, 38, 28,  1, 39, 30,  1,
        39, 50,  9,  1, 39, 50, 37, 25,  1, 39])


In [9]:
train_size = int(0.8 * len(data))
train_data = data[:train_size]
val_data = data[train_size:]

def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data)-BLOCK_SIZE, (BATCH_SIZE,))
    print(ix)
    x = torch.stack([data[i:i+BLOCK_SIZE]for i in ix])
    y = torch.stack([data[i+1:i+BLOCK_SIZE+1]for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch(split="train")
print(f'Inputs: {x} targets: {y}')

tensor([ 26652,  75324, 184989,  86537])
Inputs: tensor([[37, 29,  1, 39, 38,  9,  1, 34],
        [60,  1, 74, 72,  1, 73, 68, 76],
        [73, 62, 66, 58,  0, 54, 71, 71],
        [54, 62, 66, 58, 57, 22,  0,  0]], device='cuda:0') targets: tensor([[29,  1, 39, 38,  9,  1, 34, 33],
        [ 1, 74, 72,  1, 73, 68, 76, 54],
        [62, 66, 58,  0, 54, 71, 71, 62],
        [62, 66, 58, 57, 22,  0,  0,  3]], device='cuda:0')


In [ ]:
x = train_data[:BLOCK_SIZE]
y = train_data[1:BLOCK_SIZE+1]

for i in range(BLOCK_SIZE):
    context = x[:i+1]
    target = y[i]
    print(f'When input is {context}, target is {target}')

When input is tensor([80]), target is 28
When input is tensor([80, 28]), target is 39
When input is tensor([80, 28, 39]), target is 42
When input is tensor([80, 28, 39, 42]), target is 39
When input is tensor([80, 28, 39, 42, 39]), target is 44
When input is tensor([80, 28, 39, 42, 39, 44]), target is 32
When input is tensor([80, 28, 39, 42, 39, 44, 32]), target is 49
When input is tensor([80, 28, 39, 42, 39, 44, 32, 49]), target is 1


In [ ]:
from torch import nn

vocab_size = 10000
embedding_size = 100
embedding = nn.Embedding(vocab_size, embedding_size)

input_indicies = torch.LongTensor([1, 5, 3, 2])
output = embedding(input_indicies)

output.shape

torch.Size([4, 100])